# script to make battleground stats for php

1. Installing Chart.js

In [ ]:
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>

2. Python Code for Processing Stats

You'll need to aggregate stats for each battleground—how many times a player won, lost, and played. Here's a Python script that processes match results and prepares the data:

In [ ]:
import pymysql
import json

# Connect to the MariaDB database
conn = pymysql.connect(
    host="localhost",  # Change this if needed
    user="root",
    password="",
    database="wow"
)
cursor = conn.cursor()

# Query battleground stats per character using the correct columns and grouping
cursor.execute("""
    SELECT b.name AS battleground, c.name AS character_name, 
           SUM(cb.played) AS games_played, 
           SUM(cb.won) AS wins,
           SUM(cb.lost) AS losses
    FROM character_battlegrounds cb
    JOIN battlegrounds b ON cb.battleground_id = b.id
    JOIN characters c ON cb.character_id = c.id
    GROUP BY b.name, c.name;
""")
data = cursor.fetchall()

# Convert the data to JSON format for Chart.js  
# Here we explicitly convert Decimal values to float.
stats = [{
    "battleground": row[0],
    "character_name": row[1],
    "played": float(row[2]),
    "wins": float(row[3]),
    "losses": float(row[4])
} for row in data]

# Save JSON output for frontend use
with open("bg_stats.json", "w") as f:
    json.dump(stats, f, indent=4)

conn.close()
print("Battleground stats saved to bg_stats.json")


3. Integrating the Chart into Your Page

Modify your main content section to dynamically load stats:

In [ ]:
<div class="w3-container">
    <canvas id="bgChart"></canvas>
</div>

<script>
    fetch("bg_stats.json")
        .then(response => response.json())
        .then(data => {
            const battlegrounds = data.map(d => d.battleground);
            const wins = data.map(d => d.wins);
            const losses = data.map(d => d.losses);

            new Chart(document.getElementById("bgChart"), {
                type: "bar",
                data: {
                    labels: battlegrounds,
                    datasets: [
                        { label: "Wins", data: wins, backgroundColor: "green" },
                        { label: "Losses", data: losses, backgroundColor: "red" }
                    ]
                }
            });
        });
</script>


4. Adjusting Navigation

If you'd rather list battlegrounds in the sidebar, you could dynamically load them from the database like this:

In [ ]:
$bgNames = $db->query("SELECT name FROM battlegrounds");
foreach ($bgNames as $bg) {
    echo "<a class='list-group-item' href='#'>{$bg['name']}</a>";
}

1. Matplotlib

## Stacked Bar Chart using Matplotlib

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np

# Load the stats from JSON (assuming bg_stats.json was generated by your script)
with open("bg_stats.json", "r") as f:
    stats = json.load(f)

# For demonstration, let's aggregate the data by battleground;
# if multiple characters per battleground, we'll sum their numbers.
aggregated = {}
for record in stats:
    bg = record["battleground"]
    if bg not in aggregated:
        aggregated[bg] = {"games": 0, "wins": 0, "losses": 0}
    aggregated[bg]["games"] += record.get("played", 0)
    aggregated[bg]["wins"] += record.get("wins", 0)
    aggregated[bg]["losses"] += record.get("losses", 0)

# Prepare lists for plotting
battlegrounds = list(aggregated.keys())
wins = [aggregated[bg]["wins"] for bg in battlegrounds]
losses = [aggregated[bg]["losses"] for bg in battlegrounds]

x = np.arange(len(battlegrounds))
width = 0.35

fig, ax = plt.subplots(figsize=(10,6))
rects1 = ax.bar(x - width/2, wins, width, label="Wins", color="#198754")
rects2 = ax.bar(x + width/2, losses, width, label="Losses", color="#dc3545")

ax.set_ylabel("Count")
ax.set_title("Battleground Performance")
ax.set_xticks(x)
ax.set_xticklabels(battlegrounds, rotation=45, ha="right")
ax.legend()

# Add labels on the bars
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f"{height}",
                    xy=(rect.get_x() + rect.get_width()/2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha="center", va="bottom")

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()


2. Plotty

Grouped Bar Chart using Plotly

In [ ]:
import json
import pandas as pd
import plotly.express as px

# Load the JSON data
with open("bg_stats.json", "r") as f:
    stats = json.load(f)

# If you want to use Plotly, convert the stats into a pandas DataFrame.
df = pd.DataFrame(stats)

# Aggregate data by battleground if needed.
agg_df = df.groupby("battleground", as_index=False).agg({
    'played': 'sum',
    'wins': 'sum',
    'losses': 'sum'
})

# Create an interactive grouped bar chart
fig = px.bar(agg_df, x="battleground", y=["wins", "losses"],
             barmode="group",
             title="Battleground Performance",
             labels={"value": "Count", "battleground": "Battleground"},
             color_discrete_map={"wins": "#198754", "losses": "#dc3545"})
fig.show()


### charts.php

Loads wow_stats.json

Builds a sidebar of clickable battleground names (each link passes the battleground name as a GET parameter and opens in a new tab)

Aggregates overall stats per battleground (games played, wins, losses)

Renders an overall Chart.js chart

In [ ]:
<?php
// File: wow_stats/charts.php

// Load and decode the JSON file
$jsonData = file_get_contents("wow_stats.json");
$rawData = json_decode($jsonData, true);

// Re-structure the JSON into an array keyed by table name
$parsed = [];
foreach ($rawData as $entry) {
    if (isset($entry['type']) && $entry['type'] === 'table') {
        // Use the table name as the key and store its "data" array
        if (isset($entry['name']) && isset($entry['data'])) {
            $parsed[$entry['name']] = $entry['data'];
        }
    }
}

// Ensure we have the necessary tables
if (!isset($parsed['battlegrounds'], $parsed['character_battlegrounds'])) {
    die("Required tables not found in JSON data.");
}

// Build a mapping of battleground id => battleground name
$bgMap = [];
foreach ($parsed['battlegrounds'] as $bgRow) {
    $bgMap[$bgRow['id']] = $bgRow['name'];
}

// Build an array of battleground names (for sidebar links)
$battlegroundNames = array_values($bgMap);
sort($battlegroundNames);

// Aggregate overall stats (wins, losses, played) per battleground (by id)
$aggregated = [];
foreach ($parsed['character_battlegrounds'] as $entry) {
    $bgId = $entry['battleground_id'];
    if (!isset($aggregated[$bgId])) {
        $aggregated[$bgId] = ['wins' => 0, 'losses' => 0, 'played' => 0];
    }
    $aggregated[$bgId]['wins']   += intval($entry['won']);
    $aggregated[$bgId]['losses'] += intval($entry['lost']);
    $aggregated[$bgId]['played'] += intval($entry['played']);
}

// Reindex aggregated data using battleground names as keys
$aggByName = [];
foreach ($aggregated as $bgId => $stats) {
    $bgName = isset($bgMap[$bgId]) ? $bgMap[$bgId] : "Unknown";
    $aggByName[$bgName] = $stats;
}

// Prepare data for Chart.js
$labels = array_keys($aggByName);
$wins   = array_map(function($stats) { return $stats['wins']; }, $aggByName);
$losses = array_map(function($stats) { return $stats['losses']; }, $aggByName);
$played = array_map(function($stats) { return $stats['played']; }, $aggByName);
?>
<!DOCTYPE html>
<html lang="en">
<head>
   <meta charset="UTF-8" />
   <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
   <title>WoW Battleground Stats</title>
   <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
   <link rel="stylesheet" href="https://www.w3schools.com/w3css/5/w3.css">
   <link href="../globalStyles.css" rel="stylesheet" />
   <style>
       body { font-family: Arial, sans-serif; margin: 20px; }
       .sidebar { margin-bottom: 20px; }
       .sidebar ul { list-style: none; padding: 0; }
       .sidebar li { margin: 5px 0; }
       .sidebar a { text-decoration: none; color: blue; }
   </style>
</head>
<body>
<div class="w3-row">
    <!-- Sidebar: list of battlegrounds -->
    <div class="w3-col m3 sidebar">
         <h3>Battlegrounds</h3>
         <ul>
            <?php foreach ($battlegroundNames as $bgNameItem): ?>
               <li>
                   <a href="bg_details.php?bg=<?php echo urlencode($bgNameItem); ?>" target="_blank">
                       <?php echo htmlspecialchars($bgNameItem); ?>
                   </a>
               </li>
            <?php endforeach; ?>
         </ul>
    </div>
    <!-- Main content: Overall Aggregated Stats Chart -->
    <div class="w3-col m9">
         <h2>Overall Battleground Stats</h2>
         <canvas id="overallChart"></canvas>
         <script>
            const ctx = document.getElementById("overallChart").getContext("2d");
            new Chart(ctx, {
                type: 'bar',
                data: {
                    labels: <?php echo json_encode($labels); ?>,
                    datasets: [
                        { label: "Wins", data: <?php echo json_encode($wins); ?>, backgroundColor: "green" },
                        { label: "Losses", data: <?php echo json_encode($losses); ?>, backgroundColor: "red" },
                        { label: "Played", data: <?php echo json_encode($played); ?>, backgroundColor: "blue" }
                    ]
                },
                options: { responsive: true, scales: { y: { beginAtZero: true } } }
            });
         </script>
    </div>
</div>
</body>
</html>


### bg_details.php

When a battleground link is clicked, this page reads the GET parameter (the battleground name), looks up its ID, then filters the stats for that battleground to aggregate each character’s performance. Finally, it renders a Chart.js bar chart of that battleground’s character stats.

In [ ]:
<?php
// File: wow_stats/bg_details.php

// Get the battleground name from the query string
$bgNameParam = isset($_GET['bg']) ? trim($_GET['bg']) : '';
if (!$bgNameParam) {
    die("No battleground specified.");
}

// Load and decode the JSON file
$jsonData = file_get_contents("wow_stats.json");
$rawData = json_decode($jsonData, true);

// Build a re-structured array keyed by table name
$parsed = [];
foreach ($rawData as $entry) {
    if (isset($entry['type']) && $entry['type'] === 'table' && isset($entry['name'], $entry['data'])) {
        $parsed[$entry['name']] = $entry['data'];
    }
}

// Ensure the expected tables exist
if (!isset($parsed['battlegrounds'], $parsed['character_battlegrounds'], $parsed['characters'])) {
    die("JSON data missing required tables.");
}

// Create mappings:
// Battleground ID ⇒ Name and reverse mapping: Name ⇒ ID
$bgMap = $bgReverse = [];
foreach ($parsed['battlegrounds'] as $bgRow) {
    $bgMap[$bgRow['id']] = $bgRow['name'];
    $bgReverse[$bgRow['name']] = $bgRow['id'];
}

// Check that the requested battleground exists
if (!isset($bgReverse[$bgNameParam])) {
    die("Battleground '" . htmlspecialchars($bgNameParam) . "' not found.");
}
$bgId = $bgReverse[$bgNameParam];

// Filter character_battlegrounds records for this battleground
$filtered = [];
foreach ($parsed['character_battlegrounds'] as $entry) {
    if (intval($entry['battleground_id']) === intval($bgId)) {
        $filtered[] = $entry;
    }
}

// Build a mapping from character id to name
$charMap = [];
foreach ($parsed['characters'] as $charRow) {
    $charMap[$charRow['id']] = $charRow['name'];
}

// Aggregate stats per character for the selected battleground
$characterStats = [];
foreach ($filtered as $entry) {
    $charId = $entry['character_id'];
    $charName = isset($charMap[$charId]) ? $charMap[$charId] : "Unknown";
    if (!isset($characterStats[$charName])) {
        $characterStats[$charName] = ['played' => 0, 'wins' => 0, 'losses' => 0];
    }
    $characterStats[$charName]['played'] += intval($entry['played']);
    $characterStats[$charName]['wins']   += intval($entry['won']);
    $characterStats[$charName]['losses'] += intval($entry['lost']);
}

$labels = array_keys($characterStats);
$played = array_map(function($stats) { return $stats['played']; }, $characterStats);
$wins   = array_map(function($stats) { return $stats['wins']; }, $characterStats);
$losses = array_map(function($stats) { return $stats['losses']; }, $characterStats);
?>
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title><?php echo htmlspecialchars($bgNameParam); ?> - Character Stats</title>
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <link href="../globalStyles.css" rel="stylesheet" />
    <style>
         body { font-family: Arial, sans-serif; margin: 20px; }
         .container { width: 90%; margin: auto; }
    </style>
</head>
<body>
  <div class="container">
      <h1><?php echo htmlspecialchars($bgNameParam); ?> - Character Stats</h1>
      <canvas id="bgChart"></canvas>
  </div>
  <script>
      const labels = <?php echo json_encode($labels); ?>;
      const played = <?php echo json_encode($played); ?>;
      const wins = <?php echo json_encode($wins); ?>;
      const losses = <?php echo json_encode($losses); ?>;
      
      new Chart(document.getElementById("bgChart"), {
          type: 'bar',
          data: {
              labels: labels,
              datasets: [
                  { label: "Played", data: played, backgroundColor: "blue" },
                  { label: "Wins", data: wins, backgroundColor: "green" },
                  { label: "Losses", data: losses, backgroundColor: "red" }
              ]
          },
          options: {
              responsive: true,
              scales: { y: { beginAtZero: true } }
          }
      });
  </script>
</body>
</html>
